In [12]:
epochs = 50

# Part 7 - Aprendizado federado com Base de Dados Federado

Aqui introduzimos uma nova ferramenta para o uso de base de dados (datasets) federados. Nós criamos a classe `FederatedDataset`cuja intenção é ser usada como a classe Dataset do Pytorch, que é repassado para um carregador de dados federado `FederatedDataLoader` no qual irá iterar sobre isso em um estilo federado

Autores:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Théo Ryffel - GitHub: [@LaRiffle](https://github.com/LaRiffle)

Tradutor:
- João Lucas - GitHub: [@joaolcaas](https://github.com/joaolcaas)


Usaremos o sandbox que foi discutido no último tutorial

In [13]:
import torch as th
import syft as sy
sy.create_sandbox(globals(), verbose=False)

Setting up Sandbox...
Done!


Procure pelo dataset

In [14]:
boston_data = grid.search("#boston", "#data")
boston_target = grid.search("#boston", "#target")

Carrega-se um modelo e um otimizador

In [15]:
n_features = boston_data['alice'][0].shape[1]
n_targets = 1

model = th.nn.Linear(n_features, n_targets)

Aqui carregaremos os dados buscados em um `FederatedDataset`. Olhe os workers contém parte dos dados

In [16]:
# Cast the result in BaseDatasets
datasets = []
for worker in boston_data.keys():
    dataset = sy.BaseDataset(boston_data[worker][0], boston_target[worker][0])
    datasets.append(dataset)

# Build the FederatedDataset object
dataset = sy.FederatedDataset(datasets)
print(dataset.workers)
optimizers = {}
for worker in dataset.workers:
    optimizers[worker] = th.optim.Adam(params=model.parameters(),lr=1e-2)

['bob', 'theo', 'jason', 'alice', 'andy', 'jon']


Colocamos os dados em um `FederatedDataLoader` juntamente com informações específicas.

In [17]:
train_loader = sy.FederatedDataLoader(dataset, batch_size=32, shuffle=False, drop_last=False)

E finalmente nós iteramos sobre as épocas. É visível a similaridae quando comparado com o trainamento do PyTorch puro e local.

In [18]:
for epoch in range(1, epochs + 1):
    loss_accum = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        model.send(data.location)
        
        optimizer = optimizers[data.location.id]
        optimizer.zero_grad()
        pred = model(data)
        loss = ((pred.view(-1) - target)**2).mean()
        loss.backward()
        optimizer.step()
        
        model.get()
        loss = loss.get()
        
        loss_accum += float(loss)
        
        if batch_idx % 8 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tBatch loss: {:.6f}'.format(
                epoch, batch_idx, len(train_loader),
                       100. * batch_idx / len(train_loader), loss.item()))            
            
    print('Total loss', loss_accum)

Train Epoch: 1 [0/16 (0%)]	Batch loss: 748.940247
Train Epoch: 1 [8/16 (50%)]	Batch loss: 51.151550
Train Epoch: 1 [16/16 (100%)]	Batch loss: 1034.458130
Total loss 16811.58366394043
Train Epoch: 2 [0/16 (0%)]	Batch loss: 2326.067871
Train Epoch: 2 [8/16 (50%)]	Batch loss: 51.231117
Train Epoch: 2 [16/16 (100%)]	Batch loss: 1064.509888
Total loss 18240.483978271484
Train Epoch: 3 [0/16 (0%)]	Batch loss: 1244.071533
Train Epoch: 3 [8/16 (50%)]	Batch loss: 67.221535
Train Epoch: 3 [16/16 (100%)]	Batch loss: 523.781494
Total loss 9836.87712097168
Train Epoch: 4 [0/16 (0%)]	Batch loss: 1256.591309
Train Epoch: 4 [8/16 (50%)]	Batch loss: 56.861378
Train Epoch: 4 [16/16 (100%)]	Batch loss: 349.512665
Total loss 7150.3307456970215
Train Epoch: 5 [0/16 (0%)]	Batch loss: 769.756470
Train Epoch: 5 [8/16 (50%)]	Batch loss: 43.537251
Train Epoch: 5 [16/16 (100%)]	Batch loss: 227.863876
Total loss 4574.713926315308
Train Epoch: 6 [0/16 (0%)]	Batch loss: 385.063324
Train Epoch: 6 [8/16 (50%)]	Batch 

Train Epoch: 46 [0/16 (0%)]	Batch loss: 38.950951
Train Epoch: 46 [8/16 (50%)]	Batch loss: 44.898899
Train Epoch: 46 [16/16 (100%)]	Batch loss: 14.251371
Total loss 766.9138081073761
Train Epoch: 47 [0/16 (0%)]	Batch loss: 39.400421
Train Epoch: 47 [8/16 (50%)]	Batch loss: 44.418545
Train Epoch: 47 [16/16 (100%)]	Batch loss: 14.104561
Total loss 765.271416425705
Train Epoch: 48 [0/16 (0%)]	Batch loss: 39.721554
Train Epoch: 48 [8/16 (50%)]	Batch loss: 43.991951
Train Epoch: 48 [16/16 (100%)]	Batch loss: 13.960462
Total loss 764.0386564731598
Train Epoch: 49 [0/16 (0%)]	Batch loss: 39.886292
Train Epoch: 49 [8/16 (50%)]	Batch loss: 43.656456
Train Epoch: 49 [16/16 (100%)]	Batch loss: 13.822918
Total loss 762.9845123291016
Train Epoch: 50 [0/16 (0%)]	Batch loss: 39.897415
Train Epoch: 50 [8/16 (50%)]	Batch loss: 43.429821
Train Epoch: 50 [16/16 (100%)]	Batch loss: 13.693880
Total loss 761.9211814403534


# Parabéns!!! - Hora de se juntar a comunidade!

Parabéns por concluir esta etapa do tutorial! Se você gostou e gostaria de se juntar ao movimento em direção à proteção de privacidade, propriedade descentralizada e geração, demanda em cadeia, de dados em IA, você pode fazê-lo das seguintes maneiras!

### Dê-nos uma estrela em nosso repo do PySyft no GitHub

A maneira mais fácil de ajudar nossa comunidade é adicionando uma estrela nos nossos repositórios! Isso ajuda a aumentar a conscientização sobre essas ferramentas legais que estamos construindo.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Junte-se ao Slack!

A melhor maneira de manter-se atualizado sobre os últimos avanços é se juntar à nossa comunidade! Você pode fazer isso preenchendo o formulário em [http://slack.openmined.org](http://slack.openmined.org)

### Contribua com o projeto!

A melhor maneira de contribuir para a nossa comunidade é se tornando um contribuidor do código! A qualquer momento, você pode acessar a página de *Issues* (problemas) do PySyft no GitHub e filtrar por "Projetos". Isso mostrará todas as etiquetas (tags) na parte superior, com uma visão geral de quais projetos você pode participar! Se você não deseja ingressar em um projeto, mas gostaria de codificar um pouco, também pode procurar mais mini-projetos "independentes" pesquisando problemas no GitHub marcados como "good first issue".

- [Projetos do PySyft](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Etiquetados como Good First Issue](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Doar

Se você não tem tempo para contribuir com nossa base de códigos, mas ainda deseja nos apoiar, também pode se tornar um Apoiador em nosso Open Collective. Todas as doações vão para hospedagem na web e outras despesas da comunidade, como hackathons e meetups!

[Página do Open Collective do OpenMined](https://opencollective.com/openmined)